In [29]:
import warnings

warnings.filterwarnings("ignore")
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
import pandas as pd
import time
import os
import json
import numpy as np
import random


%load_ext nb_black

# selenium _ beautusoup _ request

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [40]:
def driver_operation(url, size):
    # options = webdriver.ChromeOptions()  # I would also suggest to use this instead of Options()
    # options.add_argument("--headless")
    # options.add_argument("--enable-javascript")  # To be on the safe side, although it seems to be enabled by default

    time.sleep(1)
    driver = webdriver.Chrome(executable_path="chromedriver_win32/chromedriver.exe")
    driver.maximize_window()
    driver.set_window_size(1920, 1080)
    driver.get(url)
    driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/div[5]/div/div/div[2]/div[2]/button").click()
    time.sleep(1)

    for i in range(0, size+1):
        driver.execute_script("window.scrollTo({start}*document.body.scrollHeight/{r}, {end}*document.body.scrollHeight/{r})".format(start=i, end = i+1, r=size))
        time.sleep(2)

    return driver







#scroll_to_bottom(driver)

<IPython.core.display.Javascript object>

In [47]:
def scrape_restaurant_info(driver, url):
        small_display = '/html/body/div[1]/div[1]/div/main/div[5]/ul'
        large_display = '/html/body/div[1]/div[1]/div/main/div[5]/div[2]/div[4]/ul'
        large_v2_display = '/html/body/div[1]/div[1]/div/main/div[5]/div/div[4]/ul'
                           
        #lst_check = [check_exists_by_xpath(small_display, driver), check_exists_by_xpath(large_display, driver), check_exists_by_xpath(large_v2_display, driver)]


        try: 
                df_restaurant = scrape_restaurant_small_display(driver)
        except:
                try:
                        df_restaurant = scrape_restaurant_large_display(driver)
                except:
                        try:
                                df_restaurant = scrape_restaurant_large_display_v2(driver)
                        except Exception:
                                print('Skip ' + url)
                                with open('restaurant_fail.txt', 'a') as f:
                                        f.write(url+'\n')
                                driver.close()
                                return None


        # try:
        #         if lst_check[0] is True:
        #                 df_restaurant = scrape_restaurant_small_display(driver)
        #                 print('small')
        #         elif lst_check[1] is True and lst_check[2] is False:
        #                 df_restaurant = scrape_restaurant_large_display(driver)               
        #                 print('large')
        #         elif lst_check[2] is True and lst_check[1] is False:
        #                 df_restaurant = scrape_restaurant_large_display_v2(driver)
        #                 print('large v2')
        #         elif lst_check[1] is True and lst_check[2] is True:
        #                 df_restaurant = scrape_restaurant_large_display(driver)               


        # except Exception:
        #         print('Skip ' + url)
        #         with open('restaurant_fail.txt', 'a') as f:
        #                 f.write(url+'\n')
        #         driver.close()
        #         return None

        driver.close()
        return df_restaurant

<IPython.core.display.Javascript object>

In [42]:

def scrape_restaurant_small_display(driver):
        small_display = '/html/body/div[1]/div[1]/div/main/div[5]/ul'
        df_restaurant = pd.DataFrame(columns = ['title','detail','rating','num_review', 'menu_category', 'name', 'price', 'description', 'status',  'img_url'])

        # ===== Header details =====
        title = ''
        detail = ''
        rating = ''
        num = ''

        try:  
                detail = driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/main/div[4]/div/div[1]/div").text

        except:
                detail = ''


        try:
                rating= driver.find_element_by_xpath(
                "/html/body/div[1]/div[1]/div/main/div[3]/div/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/div[1]").text
        except:
                rating = ''


        try:
                num = driver.find_element_by_xpath(
                "/html/body/div[1]/div[1]/div/main/div[3]/div/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/div[3]").text
        except:
                num = ''


        try:
                title =  driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/main/div[3]/div/div[2]/div[3]/div[1]/div[2]/div[2]/h1").text,

        except:
                title = ''


        # menu
        list_item_element = driver.find_element_by_xpath(small_display).find_element_by_tag_name("li")
        class_name = list_item_element.get_attribute("class").replace(' ','.')
        menu = driver.find_element_by_xpath(small_display).find_elements_by_class_name(class_name)



        for x in range(1, len(menu)+1): 
                cat_pos = "/html/body/div[1]/div[1]/div/main/div[5]/ul/li[" + str(x) + "]/h2"
                category = driver.find_element_by_xpath(cat_pos).text 

                # restaurant['menu'].append({
                #         category: []
                # })
                section = driver.find_element_by_xpath(
                        "/html/body/div[1]/div[1]/div/main/div[5]/ul/li[" + str(x) + "]/ul").find_elements_by_tag_name("li")
                        
                for y in range(1, len(section)+1):

                        # Get Product Name
                        try:  
                                name = str(driver.find_element_by_xpath(
                                        "/html/body/div[1]/div[1]/div/main/div[5]/ul/li[" + str(x) + "]/ul/li[" + str(
                                        y) + "]/div/div/div/div[1]/div[1]/h4/div").text)
                        except:
                                name = ''

                        ## Get Product Description
                        try:

                                description = str(driver.find_element_by_xpath(
                                        "/html/body/div[1]/div[1]/div/main/div[5]/ul/li[" + str(x) + "]/ul/li[" + str(
                                        y) + "]/div/div/div/div[1]/div[2]/div").text)

                        except:
                                description = ''

                        # Get Product Price
                        try:
                                price = str(driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/main/div[5]/ul/li[" + str(x) + "]/ul/li[" + str(
                                        y) + "]/div/div/div/div[1]/div[3]/div").text)

                                status = "In stock"
                                if price == description:
                                        description = ''

                                if "Sold" in price:
                                        status = "Sold out"
                                        price = "$" + price.split("$", 1)[1]
                                else:
                                        status = "In stock"
                        except:

                                if "$" in description:
                                        price = description
                                        description = ''
                                else:
                                        price = ''
                                        status = "In stock"
                        
                        # Get Image URL  
                        
                        
                        try:  
                                img_url = str(driver.find_element_by_xpath(
                                        "/html/body/div[1]/div[1]/div/main/div[5]/ul/li[" + str(x) + "]/ul/li[" + str(\
                                        y) + "]/div/div/div/div[2]/div/picture/img").get_attribute("src"))


                        
                        except:
                                img_url = ''


                        try:
                                df_restaurant = df_restaurant.append({
                                'title': title,
                                'detail': detail,
                                'rating': rating,
                                'num_review': num,
                                'menu_category': category,
                                'name': name,
                                'price': price,
                                'description': description,
                                'status': status,
                                'img_url': img_url
                                }, ignore_index=True)
                        except:
                                df_restaurant = df_restaurant.append({
                                'title': title,
                                'detail': detail,
                                'rating': rating,
                                'num_review': num,
                                'menu_category': category,
                                'name': name,
                                'price': price,
                                'description': description,
                                'img_url': img_url
                                }, ignore_index=True)
                
        return df_restaurant


<IPython.core.display.Javascript object>

In [43]:

def scrape_restaurant_large_display(driver):
        df_restaurant = pd.DataFrame(columns = ['title','detail','rating','num_review', 'menu_category', 'name', 'price', 'description', 'status',  'img_url'])
        large_display = '/html/body/div[1]/div[1]/div/main/div[5]/div[2]/div[4]/ul'

        
        # ===== Header details =====
        title = ''
        detail = ''
        rating = ''
        num = ''

        try:  
                detail = driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/main/div[4]/div/div[1]/div").text

        except:
                detail = ''


        try:
                rating= driver.find_element_by_xpath(
                "/html/body/div[1]/div[1]/div/main/div[3]/div/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/div[1]").text
        except:
                rating = ''


        try:
                num = driver.find_element_by_xpath(
                "/html/body/div[1]/div[1]/div/main/div[3]/div/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/div[3]").text
        except:
                num = ''


        try:
                title =  driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/main/div[3]/div/div[2]/div[3]/div[1]/div[2]/div[2]/h1").text,

        except:
                title = ''


        # menu
        list_item_element = driver.find_element_by_xpath(large_display).find_element_by_tag_name("li")
        class_name = list_item_element.get_attribute("class").replace(' ','.')
        menu = driver.find_element_by_xpath(large_display).find_elements_by_class_name(class_name)

        for x in range(1, len(menu)+1): 
                cat_pos = "/html/body/div[1]/div[1]/div/main/div[5]/div[2]/div[4]/ul/li[" + str(x) + "]/div[1]"
                category = driver.find_element_by_xpath(cat_pos).text 
                # restaurant['menu'].append({
                #         category: []
                # })

                section = driver.find_element_by_xpath(
                        "/html/body/div[1]/div[1]/div/main/div[5]/div[2]/div[4]/ul/li[" + str(x) + "]/ul").find_elements_by_tag_name("li")
                        
                for y in range(1, len(section)+1):

                        # Get Product Name
                        try:  
                                name = str(driver.find_element_by_xpath(
                                        "/html/body/div[1]/div[1]/div/main/div[5]/div[2]/div[4]/ul/li[" + str(x) + "]/ul/li[" + str(
                                        y) + "]/div/div/div[2]/div[1]/span").text)
                        except:
                                name = ''

                        ## Get Product Description
                        try:

                                description = str(driver.find_element_by_xpath(
                                        "/html/body/div[1]/div[1]/div/main/div[5]/div[2]/div[4]/ul/li[" + str(x) + "]/ul/li[" + str(
                                        y) + "]/div/div/div[2]/div[1]/span").text)

                        except:
                                description = ''

                        # Get Product Price
                        try:
                                   
                                price = str(driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/main/div[5]/div[2]/div[4]/ul/li[" + str(x) + "]/ul/li[" + str(
                                        y) + "]/div/div/div[2]/div[2]/span").text)

                                if price == description:
                                        description = ''

                                if "Sold" in price:
                                        status = "Sold out"
                                        price = "$" + price.split("$", 1)[1]
                                else:
                                        status = "In stock"
                        except:

                                if "$" in description:
                                        price = description
                                        description = ''
                                else:
                                        price = ''
                        
                        # Get Image URL  
                        try:  
                                img_url = str(driver.find_element_by_xpath(
                                        "/html/body/div[1]/div[1]/div/main/div[5]/div[2]/div[4]/ul/li[" + str(x) + "]/ul/li[" + str(\
                                        y) + "]/div/div/div[1]/div/picture/img").get_attribute("src"))
        
                        except:
                                img_url = ''

                        try:
                                df_restaurant = df_restaurant.append({
                                'title': title,
                                'detail': detail,
                                'rating': rating,
                                'num_review': num,
                                'menu_category': category,
                                'name': name,
                                'price': price,
                                'description': description,
                                'status': status,
                                'img_url': img_url
                                }, ignore_index=True)
                        except:
                                df_restaurant = df_restaurant.append({
                                'title': title,
                                'detail': detail,
                                'rating': rating,
                                'num_review': num,
                                'menu_category': category,
                                'name': name,
                                'price': price,
                                'description': description,
                                'img_url': img_url
                                }, ignore_index=True)
                


        return df_restaurant


<IPython.core.display.Javascript object>

In [44]:

def scrape_restaurant_large_display_v2(driver):
        df_restaurant = pd.DataFrame(columns = ['title','detail','rating','num_review', 'menu_category', 'name', 'price', 'description', 'status',  'img_url'])
        large_v2_display = '/html/body/div[1]/div[1]/div/main/div[5]/div/div[4]/ul'

        
        # ===== Header details =====
        title = ''
        detail = ''
        rating = ''
        num = ''

        try:  
                detail = driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/main/div[4]/div/div[1]/div").text

        except:
                detail = ''


        try:
                rating= driver.find_element_by_xpath(
                "/html/body/div[1]/div[1]/div/main/div[3]/div/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/div[1]").text
        except:
                rating = ''


        try:
                num = driver.find_element_by_xpath(
                "/html/body/div[1]/div[1]/div/main/div[3]/div/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/div[3]").text
        except:
                num = ''


        try:
                title =  driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/main/div[3]/div/div[2]/div[3]/div[1]/div[2]/div[2]/h1").text,

        except:
                title = ''


        # menu
        list_item_element = driver.find_element_by_xpath(large_v2_display).find_element_by_tag_name("li")
        class_name = list_item_element.get_attribute("class").replace(' ','.')
        menu = driver.find_element_by_xpath(large_v2_display).find_elements_by_class_name(class_name)

        for x in range(1, len(menu)+1): 
                cat_pos = "/html/body/div[1]/div[1]/div/main/div[5]/div/div[4]/ul/li[" + str(x) + "]/div[1]"
                category = driver.find_element_by_xpath(cat_pos).text 
                # restaurant['menu'].append({
                #         category: []
                # })
                section = driver.find_element_by_xpath(
                        "/html/body/div[1]/div[1]/div/main/div[5]/div/div[4]/ul/li[" + str(x) + "]/ul").find_elements_by_tag_name("li")
                        
                for y in range(1, len(section)+1):

                        # Get Product Name
                        try:  
                                name = str(driver.find_element_by_xpath(
                                        "/html/body/div[1]/div[1]/div/main/div[5]/div/div[4]/ul/li[" + str(x) + "]/ul/li[" + str(
                                        y) + "]/div/div/div[2]/div[1]/span").text)
                        except:
                                name = ''

                        ## Get Product Description
                        try:

                                description = str(driver.find_element_by_xpath(
                                        "/html/body/div[1]/div[1]/div/main/div[5]/div/div[4]/ul/li[" + str(x) + "]/ul/li[" + str(
                                        y) + "]/div/div/div[2]/div[1]/span").text)

                        except:
                                description = ''

                        # Get Product Price
                        try:   
                                price = str(driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/main/div[5]/div/div[4]/ul/li[" + str(x) + "]/ul/li[" + str(
                                        y) + "]/div/div/div[2]/div[2]/span").text)

                                if price == description:
                                        description = ''

                                if "Sold" in price:
                                        status = "Sold out"
                                        price = "$" + price.split("$", 1)[1]
                                else:
                                        status = "In stock"
                        except:

                                if "$" in description:
                                        price = description
                                        description = ''
                                else:
                                        price = ''
                        
                        # Get Image URL  
                        try:    
                                img_url = str(driver.find_element_by_xpath(
                                        "/html/body/div[1]/div[1]/div/main/div[5]/div/div[4]/ul/li[" + str(x) + "]/ul/li[" + str(\
                                        y) + "]/div/div/div[1]/div/picture/img").get_attribute("src"))
        
                        except:
                                img_url = ''

                        try:
                                df_restaurant = df_restaurant.append({
                                'title': title,
                                'detail': detail,
                                'rating': rating,
                                'num_review': num,
                                'menu_category': category,
                                'name': name,
                                'price': price,
                                'description': description,
                                'status': status,
                                'img_url': img_url
                                }, ignore_index=True)
                        except:
                                df_restaurant = df_restaurant.append({
                                'title': title,
                                'detail': detail,
                                'rating': rating,
                                'num_review': num,
                                'menu_category': category,
                                'name': name,
                                'price': price,
                                'description': description,
                                'img_url': img_url
                                }, ignore_index=True)
                


        return df_restaurant


<IPython.core.display.Javascript object>

# Main Test

In [1]:
with open('restaurant.txt') as f:
    lst_urls = f.read().splitlines()

len(lst_urls)


25752

In [ ]:
num = 5 # số lượng restaurant crawled, có thể đổi thành 10, 20 để test thử
# 5  restaurant crawled chạy trong khoảng từ 4-5 phút
# 10 thì khoảng 9-10'
# thời gian chạy có thể lâu hơn do tốc độ mạng và tốc độ xử lý máy nha các bạn:)

In [48]:
df = pd.DataFrame(columns = ['title','detail','rating','num_review', 'menu_category', 'name', 'price', 'description', 'status',  'img_url'])
df.to_csv('data.csv', encoding='utf-8-sig', mode='w', index=False)
lst_urls_test = random.sample(lst_urls, num)

for url in lst_urls_test:
    driver = driver_operation(url, 14)
    df = scrape_restaurant_info(driver, url)
    if df is not None:
        df.to_csv('data.csv', encoding='utf-8-sig', mode='a', header=False, index=False)
        df = df.iloc[0:0]


<IPython.core.display.Javascript object>